In [1]:
import pandas as pd
import json

pd.set_option.max_rows = 999
pd.set_option.max_columns = 999

with open('restaurant_data.json') as f:
    data = json.load(f)

In [2]:
# Task 1.1
restaurants = []
for entry in data:
    for restaurant_entry in entry.get('restaurants', []):
        e = restaurant_entry['restaurant']
        restaurant = {
            'restaurant_id': e['id'],
            'restaurant_name': e['name'],
            'country_id': e['location']['country_id'],
            'city': e['location']['city'],
            'user_rating_votes': e['user_rating']['votes'],
            'user_aggregate_rating': e['user_rating']['aggregate_rating'],
            'cuisines': e['cuisines']
        }
        restaurants.append(restaurant)

restaurants_df = pd.DataFrame(restaurants)
print(restaurants_df)


     restaurant_id              restaurant_name  country_id       city   
0         18649486         The Drunken Botanist           1    Gurgaon  \
1           308322             Hauz Khas Social           1  New Delhi   
2         18856789         AIR- An Ivory Region           1  New Delhi   
3           307374                     AMA Cafe           1  New Delhi   
4         18238278                      Tamasha           1  New Delhi   
...            ...                          ...         ...        ...   
1295       7900186                       Mexico         148   Hamilton   
1296       7900182           Thai House Express         148   Hamilton   
1297       7900023                  Indian Star         148   Hamilton   
1298       7900490           Zealong Tea Estate         148   Hamilton   
1299       7900115  The Flying Burrito Brothers         148   Hamilton   

     user_rating_votes user_aggregate_rating   
0                 4765                   4.4  \
1              

In [3]:
country_code_df = pd.read_excel('Country-Code.xlsx')

merged_df = pd.merge(restaurants_df, country_code_df, how='left', left_on='country_id', right_on='Country Code')

merged_df.drop(columns=['country_id', 'Country Code'], inplace=True)
merged_df.to_csv('restaurants.csv', index=False)

In [8]:
# Task 1.2
restaurant_events = []
for entry in data:
    for restaurant_entry in entry.get('restaurants', []):
        e = restaurant_entry['restaurant']
        for zomato_event in e.get('zomato_events', []):
            event = zomato_event['event']
            for photos in event.get('photos', []):
                photo = photos['photo']
                
                restaurant_event = {
                    'event_id': event['event_id'],
                    'restaurant_id': e['id'],
                    'restaurant_name': e['name'],
                    'photo_url': photo['url'],
                    'event_title': event['title'],
                    'event_start_date': event['start_date'],
                    'event_end_date': event['end_date']
                }

                restaurant_events.append(restaurant_event)

restaurant_events_df = pd.DataFrame(restaurant_events)

# Convert event_start_date and event_end_date to datetime objects
restaurant_events_df['event_start_date'] = pd.to_datetime(restaurant_events_df['event_start_date'])
restaurant_events_df['event_end_date'] = pd.to_datetime(restaurant_events_df['event_end_date'])

# Start date <= April 2019 and end date >= April 2019
filter = (restaurant_events_df['event_start_date'].dt.month <= 4) & (restaurant_events_df['event_start_date'].dt.year <= 2019) & \
       (restaurant_events_df['event_end_date'].dt.month >= 4) & (restaurant_events_df['event_end_date'].dt.year >= 2019)

restaurant_events = restaurant_events_df[filter]

print(restaurant_events.isna().sum())
print(restaurant_events)

event_id            0
restaurant_id       0
restaurant_name     0
photo_url           0
event_title         0
event_start_date    0
event_end_date      0
dtype: int64
     event_id restaurant_id                 restaurant_name   
0      322331      18649486            The Drunken Botanist  \
1      332812        308322                Hauz Khas Social   
2      336644      18856789            AIR- An Ivory Region   
3      336890      18382360                           Local   
4      336894      18382360                           Local   
..        ...           ...                             ...   
201    305850       7001670  Eight - Cordis, Auckland Hotel   
202    298162       7001670  Eight - Cordis, Auckland Hotel   
203    329300      18981544                        Katsu YA   
204    329301      18981544                        Katsu YA   
205    335217      18680113            The Cutt Grill House   

                                             photo_url   
0    https://b.zmt

In [7]:
restaurant_events.to_csv('restaurant_events.csv', index=False)

In [13]:
# Task 1.3
user_ratings = []
for entry in data:
    for restaurant_entry in entry.get('restaurants', []):
        e = restaurant_entry['restaurant']
        rating = e['user_rating']
        user_rating = {
            'aggregate_rating': rating['aggregate_rating'],
            'rating_text': rating['rating_text']
        }
        user_ratings.append(user_rating)

user_ratings_df = pd.DataFrame(user_ratings)
print(user_ratings_df)


     aggregate_rating rating_text
0                 4.4   Very Good
1                 4.6   Excellent
2                 4.1   Very Good
3                 4.4   Very Good
4                 4.4   Very Good
...               ...         ...
1295              4.2   Very Good
1296              4.0   Very Good
1297              3.8        Good
1298              4.7   Excellent
1299              3.9        Good

[1300 rows x 2 columns]


In [21]:
# Find Threshold
selected_ratings = ['Excellent', 'Very Good', 'Good', 'Average', 'Poor']

filtered_user_ratings_df = user_ratings_df[user_ratings_df['rating_text'].isin(selected_ratings)]

rating_ranges = filtered_user_ratings_df.groupby('rating_text')['aggregate_rating'].agg(['min', 'max'])

print(rating_ranges)

             min  max
rating_text          
Average      2.5  3.4
Excellent    4.5  4.9
Good         3.5  3.9
Poor         2.2  2.2
Very Good    4.0  4.4


Rating Thresholds:
- Excellent: 4.5
- Very Good: 4.0
- Good: 3.5
- Average: 2.5
- Poor: 2.2